In [1]:
import pandas as pd
import numpy as np
import json
import datetime as dt

### Station
??? - region 0 //
Europe - region 1 //
Oceania - region 2 //
N America - region 3 //
Asia - region 4 //
C/S America - region 5 //
Africa - region 6 //
Japan - region 7

In [2]:
lat=49.3996597
lon=15.5970656
region=1

start=dt.datetime(2020,5,11,16,0)
end=dt.datetime(2020,5,11,18,0)

import os, time
os.environ['TZ'] = 'UTC'
time.tzset()

#print(dt.datetime.now())

start=int(start.timestamp())*1000000000
end=int(end.timestamp())*1000000000

start=pd.to_datetime(start, unit='ns')
end=pd.to_datetime(end, unit='ns')
print(start,end)

2020-05-12 10:45:03.087776
2020-05-11 16:00:00 2020-05-11 18:00:00


In [3]:
### url for json data preparation
'''print(pd.to_datetime(start, unit='ns'))
print(pd.to_datetime(next, unit='ns'))
print(pd.to_datetime(end, unit='ns'))
print(np.ceil((end-start).total_seconds()/600))'''
url=[]
for i in range(0,int(np.ceil((end-start).total_seconds()/600)+1)):
    next=start+ pd.Timedelta(minutes=i*10)
    url.append('http://data.blitzortung.org/Data/Restricted/Strikes_%01d/%02d/%02d/%02d/%02d/%02d.json' % (region, next.year, next.month, next.day, next.hour, np.floor((next.minute)//10)*10))

In [4]:
import requests
blitz=pd.DataFrame()
for i in range(len(url)):
    try:
        html = requests.get(url[i]).content
        blitz_part = pd.read_json(html,lines=True)
        blitz_part.drop(columns=['status','sig'], inplace=True)#,'region', 
        blitz=blitz.append(blitz_part)
    except:
        print("No data", url[i])
blitz.set_index(['time'], inplace=True)
blitz

,lat,lon,alt,pol,mds,mcg,region
time,,,,,,,
1589212801136250382,52.270369,20.510259,0,0,6668,131,1
1589212801136245117,52.280896,20.562023,0,0,9065,186,1
1589212801677906650,49.576910,15.677801,0,0,7361,101,1
1589212801946362393,49.447060,15.733651,0,0,8199,59,1
1589212802457473281,49.407937,15.587958,0,0,6850,143,1
...,...,...,...,...,...,...,...
1589220595735588785,46.046564,12.865681,0,0,12579,96,1
1589220595911255769,45.804165,12.358225,0,0,10945,118,1
1589220595911266518,45.900227,12.371352,0,0,5462,99,1


In [10]:
### reduction of too distant events
blitz=blitz[(blitz['lat']>(lat-1) )&(blitz['lat']<(lat+1) )]
blitz=blitz[(blitz['lon']>(lon-1) )&(blitz['lon']<(lon+1) )]
#print(blitz)
### calculation of distance of events from station
from math import radians, cos, sin, asin, sqrt, atan2, degrees

def haversine2(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2.)**2. + cos(lat1) * cos(lat2) * sin(dlon/2.)**2.
    c = 2. * asin(sqrt(a)) 
    km = 6367. * c
    return km
 # with function
'''blitz['distance']=100
for i in range(len(blitz['lat'])):
    blitz['distance'][i]=haversine2(blitz.lon.iloc[i],blitz.lat.iloc[i],lon,lat)'''
    
#with pandas
blitz['lonRad']=np.radians(blitz.lon)
blitz['latRad']=np.radians(blitz.lat)
lonRad=np.radians(lon)
latRad=np.radians(lat)
blitz['DlonRad']=lonRad-blitz.lonRad
blitz['DlatRad']=latRad-blitz.latRad
blitz['a']=np.sin(blitz.DlatRad/2.)**2. + np.cos(blitz.latRad) * np.cos(latRad) * np.sin(blitz.DlonRad/2.)**2.
blitz['km'] = 2. * np.arcsin(np.sqrt(blitz.a))*6378.
blitz.drop(columns=['lonRad','latRad','DlonRad','DlatRad','a'], inplace=True)
try:
    blitz.drop(columns=['alt','pol','region'], inplace=True)
except:
    print('No columns to drop (alt, pol, region)')
blitz['time']=blitz.index
blitz

No columns to drop (alt, pol, region)


,lat,lon,mds,mcg,km,time
time,,,,,,
1589212801677906650,49.576910,15.677801,7361,101,20.576580,1589212801677906650
1589212801946362393,49.447060,15.733651,8199,59,11.209380,1589212801946362393
1589212802457473281,49.407937,15.587958,6850,143,1.133236,1589212802457473281
1589212802526827896,49.405487,15.593112,6768,71,0.709087,1589212802526827896
1589212802526824901,49.421077,15.604264,7075,80,2.440451,1589212802526824901
...,...,...,...,...,...,...
1589219171060928622,48.405994,16.583480,13495,97,132.076044,1589219171060928622
1589219171152579572,48.425541,16.593938,7584,72,130.676828,1589219171152579572
1589219553664241706,48.460824,16.569251,10354,152,126.397825,1589219553664241706


In [22]:
df = blitz

import plotly.express as px
'''fig = px.scatter_geo(df, lon="lon", lat='lat' ,color="time",
                     hover_name="km",
                    projection="natural earth"
                    )'''
fig = px.scatter(df, x="lon", y='lat' ,color="time",
                     hover_name="km",
                    )
fig.show()
fig.write_html("./blitzortung/blesky.html")